In [15]:
import os
from util import parse_kfold_results
from util import find_optimal_epoch
from util import analyze_convergence
from util import print_recommendations

In [16]:
spe = "yeast1"
models = ["DeepFE-PPI", "DeepTrio", "PIPR", "TF-PPI"]
# model2epoch = {"TF-PPI" : 50}
model2epoch = {}
optimal_epochs = {}

for model in models:
    result_file = os.path.join("result", spe, f"{model}.txt")
    if not os.path.exists(result_file):
        continue
    
    with open(result_file, "r", encoding="utf-8") as fin:
        kfold_text = fin.read()

    df = parse_kfold_results(kfold_text)

    epoch_stats, optimal_epoch = find_optimal_epoch(df, model2epoch.get(model, None))
    if optimal_epoch is None:
        continue
    
    optimal_epochs[model] = optimal_epoch

    convergence_df = analyze_convergence(df)
    
    print("=" * 80)
    print(model)
    print("=" * 80)

    print_recommendations(df, epoch_stats, optimal_epoch, convergence_df)

    print("\n")

PIPR

最优epoch: 43
--------------------------------------------------
综合评分: 0.8925

平均准确率: 89.57 (±8.70)
平均精确率: 92.41 (±11.68)
平均召回率: 88.20 (±11.80)
平均F1分数: 89.45 (±8.49)
平均ROC AUC: 89.57 (±8.71)
平均PR AUC: 87.19 (±10.94)

平均验证损失: 0.2345

收敛分析:
--------------------------------------------------
 fold  final_f1  stable_epoch  converged_early
    1    0.9559            28             True
    2    0.9608            35             True
    3    0.9449            31             True
    4    0.9552            26             True
    5    0.6681            44            False

前5个最佳epoch:
--------------------------------------------------
 epoch  composite_score  f1_mean  f1_std
    43          0.89252   0.8945  0.0849
    35          0.88457   0.8983  0.1068
    49          0.88097   0.8996  0.1225
    46          0.87830   0.8985  0.1257
    44          0.87496   0.8999  0.1297

建议:
--------------------------------------------------
建议选择epoch 43 - 模型在稳定期达到最佳性能
模型表现有一定波动，建议检查数据分布一致性


TF-PPI

In [17]:
print(f"Model\tAccuracy\tPrecision\tRecall\tF1\tROC AUC\tPR AUC\tComposite Score")
for model, optimal_epoch in optimal_epochs.items():
    accuracy = f"{optimal_epoch['accuracy_mean']*100:.2f}±{optimal_epoch['accuracy_std']*100:.2f}"
    precision = f"{optimal_epoch['precision_mean']*100:.2f}±{optimal_epoch['precision_std']*100:.2f}"
    recall = f"{optimal_epoch['recall_mean']*100:.2f}±{optimal_epoch['recall_std']*100:.2f}"
    f1 = f"{optimal_epoch['f1_mean']*100:.2f}±{optimal_epoch['f1_std']*100:.2f}"
    roc_auc = f"{optimal_epoch['roc_auc_mean']*100:.2f}±{optimal_epoch['roc_auc_std']*100:.2f}"
    pr_auc = f"{optimal_epoch['pr_auc_mean']*100:.2f}±{optimal_epoch['pr_auc_std']*100:.2f}"
    composite_score = f"{optimal_epoch['composite_score']:.4f}"
    

    print(f"{model}\t{accuracy}\t{precision}\t{recall}\t{f1}\t{roc_auc}\t{pr_auc}\t{composite_score}")


Model	Accuracy	Precision	Recall	F1	ROC AUC	PR AUC	Composite Score
PIPR	89.57±8.70	92.41±11.68	88.20±11.80	89.45±8.49	89.57±8.71	87.19±10.94	0.8925
TF-PPI	90.61±1.52	90.71±3.08	90.66±3.90	90.60±1.60	90.62±1.52	86.85±2.21	0.9064
